In [ ]:
import nannyml as nml
from IPython.display import display

reference_df, analysis_df, analysis_targets_df = nml.load_synthetic_car_loan_dataset()
display(reference_df.head())

,id,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1
3,3,22652.0,20K - 20K €,0.705992,16.0,False,10%,0.453649,1.0,2018-01-01 00:26:09.456,0.98,1
4,4,21268.0,60K+ €,0.671888,21.0,True,30%,5.695263,1.0,2018-01-01 00:34:52.608,0.99,1


In [ ]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(reference_df.head())

+----+------+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+-----------------+----------+-------------------------+----------------+----------+
|    | id   | car_value   | salary_range   | debt_to_income_ratio   | loan_length   | repaid_loan_on_prev_car   | size_of_downpayment   | driver_tenure   | repaid   | timestamp               | y_pred_proba   | y_pred   |
+====+======+=============+================+========================+===============+===========================+=======================+=================+==========+=========================+================+==========+
| 0  | 0    | 39811       | 40K - 60K €    | 0.63295                | 19            | False                     | 40%                   | 0.212653        | 1        | 2018-01-01 00:00:00.000 | 0.99           | 1        |
+----+------+-------------+----------------+------------------------+---------------+---------------------------+---

In [ ]:
feature_column_names = [
    'car_value', 'debt_to_income_ratio', 'driver_tenure'
]
calc = nml.SummaryStatsMedianCalculator(
    column_names=feature_column_names,
)

In [ ]:
calc.fit(reference_df)
results = calc.calculate(analysis_df)
display(results.filter(period='all').to_df())

chunk                                                        \
              key chunk_index start_index end_index start_date end_date   
0        [0:4999]           0           0      4999       None     None   
1     [5000:9999]           1        5000      9999       None     None   
2   [10000:14999]           2       10000     14999       None     None   
3   [15000:19999]           3       15000     19999       None     None   
4   [20000:24999]           4       20000     24999       None     None   
5   [25000:29999]           5       25000     29999       None     None   
6   [30000:34999]           6       30000     34999       None     None   
7   [35000:39999]           7       35000     39999       None     None   
8   [40000:44999]           8       40000     44999       None     None   
9   [45000:49999]           9       45000     49999       None     None   
10       [0:4999]           0           0      4999       None     None   
11    [5000:9999]           1        5000      9999       None     None   
12  [10000:14999]           2       10000     14999       None     None   
13  [15000:19999]           3       15000     19999       None     None   
14  [20000:24999]           4       20000     24999       None     None   
15  [25000:29999]           5       25000     29999       None     None   
16  [30000:34999]           6       30000     34999       None     None   
17  [35000:39999]           7       35000     39999       None     None   
18  [40000:44999]           8       40000     44999       None     None   
19  [45000:49999]           9       45000     49999       None     None   

              car_value                                           ...  \
       period     value sampling_error upper_confidence_boundary  ...   
0   reference   21985.5      285.39797              22841.693911  ...   
1   reference   21970.5      285.39797              22826.693911  ...   
2   reference   21932.0      285.39797              22788.193911  ...   
3   reference   21621.0      285.39797              22477.193911  ...   
4   reference   21642.0      285.39797              22498.193911  ...   
5   reference   21886.5      285.39797              22742.693911  ...   
6   reference   22212.5      285.39797              23068.693911  ...   
7   reference   21681.5      285.39797              22537.693911  ...   
8   reference   22191.0      285.39797              23047.193911  ...   
9   reference   21789.5      285.39797              22645.693911  ...   
10   analysis   22009.5      285.39797              22865.693911  ...   
11   analysis   22053.0      285.39797              22909.193911  ...   
12   analysis   22026.0      285.39797              22882.193911  ...   
13   analysis   21994.0      285.39797              22850.193911  ...   
14   analysis   21952.5      285.39797              22808.693911  ...   
15   analysis   43820.5      285.39797              44676.693911  ...   
16   analysis   45036.5      285.39797              45892.693911  ...   
17   analysis   45215.5      285.39797              46071.693911  ...   
18   analysis   44852.5      285.39797              45708.693911  ...   
19   analysis   44438.0      285.39797              45294.193911  ...   

   debt_to_income_ratio                        driver_tenure                 \
        upper_threshold lower_threshold  alert         value sampling_error   
0              0.665299        0.654625  False      5.642352       0.040332   
1              0.665299        0.654625  False      5.649750       0.040332   
2              0.665299        0.654625  False      5.515347       0.040332   
3              0.665299        0.654625  False      5.607354       0.040332   
4              0.665299        0.654625  False      5.590062       0.040332   
5              0.665299        0.654625  False      5.592783       0.040332   
6              0.665299        0.654625  False      5.524955       0.040332   
7              0.665299        0.654625  Fa

In [ ]:
print_multi_index_markdown(results.filter(period='all').to_df())

+----+---------------+-----------------+-----------------+---------------+----------------+--------------+------------+---------------+--------------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+--------------------------+--------------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+-------------------+--------------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                |              |            | | car_value   |                    |                               |                               |                     |                     |           | | debt_to_income_ratio   |                    |                               |                               |      

In [ ]:
for column_name in results.column_names:
    results.filter(column_names=column_name).plot().show()

In [ ]:
for column_name in results.column_names:
    results.filter(column_names=column_name).plot().write_image(
        f"../_static/tutorials/stats/median-{column_name}.svg"
    )

In [ ]:
calc1 = nml.PerformanceCalculator(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    problem_type='classification_binary',
    metrics=['roc_auc'],
    chunk_size=5000)

calc1.fit(reference_df)

analysis_df = analysis_df.merge(analysis_targets_df, left_index=True, right_index=True)
results1 = calc1.calculate(analysis_df)

ranker1 = nml.CorrelationRanker()

# ranker fits on one metric and reference period data only
ranker1.fit(
    results1.filter(period='reference', metrics=['roc_auc']))
# ranker ranks on one drift method and one performance metric
correlation_ranked_features1 = ranker1.rank(
    results,
    results1,
    only_drifting = False)

display(correlation_ranked_features1)

,column_name,pearsonr_correlation,pearsonr_pvalue,has_drifted,rank
0,car_value,0.921746,7.846795e-09,True,1
1,driver_tenure,0.168346,4.780281e-01,False,2
2,debt_to_income_ratio,0.118717,6.181305e-01,False,3
